In [1]:
import comet_ml
# Replace with your actual Comet API key
comet_api_key = "yd6X45GKLWoIKspb4dChTuwOs"
import torch 
import torch.optim as optim
import torch.nn as nn

import mitdeeplearning as mdl
import numpy as np
import time , os , functools
from IPython import display
from tqdm import tqdm
from scipy.io.wavfile import write

# Check if GPU is available, otherwise use CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


Using device: cpu


In [2]:
songs = mdl.lab1.load_training_data()

exa = songs[3]
print(exa)

Found 817 songs in text
X:4
T:Blackbird
Z: id:dc-hornpipe-4
M:C|
L:1/8
K:D Mixolydian
AG|F2FA GFD2|de (3fed d^cAF|G2GF GFDE|FdcA G2AG|!
F2FA GFD2|de (3fed d^cAG|AdcA GcAG|F2D2 D2:|!
fg|agfa gfeg|fd e^c d=cA2|agfa gfde|fdgf e2 fg|!
a2ge f3e|d^cde fdAG|AdcA GcAG|F2D2 D2:|!


In [3]:
mdl.lab1.play_song(exa)

In [9]:
songs_joined = "\n\n".join(songs)

# Find all unique characters in the joined string
vocab = sorted(set(songs_joined))
print("There are", len(vocab), "unique characters in the dataset")

There are 83 unique characters in the dataset


In [5]:
char2idx = {u: i for i, u in enumerate(vocab)}

idx2char = np.array(vocab)

In [ ]:
print('{')
for char, _ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

In [18]:
### Vectorize the songs string ###

'''TODO: Write a function to convert the all songs string to a vectorized
    (i.e., numeric) representation. Use the appropriate mapping
    above to convert from vocab characters to the corresponding indices.

  NOTE: the output of the `vectorize_string` function
  should be a np.array with `N` elements, where `N` is
  the number of characters in the input string
'''
songs_joined = "\n\n".join(songs)
def vectorize_string(string):
  vectorize_string = np.array([char2idx[char] for char in string])
  return vectorize_string

vectorized_songs = vectorize_string(songs_joined)

In [19]:
print ('{} ---- characters mapped to int ----> {}'.format(repr(songs_joined[:10]), vectorized_songs[:10]))
# check that vectorized_songs is a numpy array
assert isinstance(vectorized_songs, np.ndarray), "returned result should be a numpy array"

'X:1\nT:Alex' ---- characters mapped to int ----> [49 22 13  0 45 22 26 67 60 79]


In [20]:
### Batch definition to create training examples ###

def get_batch(vectorized_songs, seq_length, batch_size):
    # the length of the vectorized songs string
    n = vectorized_songs.shape[0] - 1
    # randomly choose the starting indices for the examples in the training batch
    idx = np.random.choice(n - seq_length, batch_size)

    '''TODO: construct a list of input sequences for the training batch'''
    input_batch = [vectorized_songs[i : i +seq_length] for i in idx]

    '''TODO: construct a list of output sequences for the training batch'''
    output_batch = [vectorized_songs[i+1 : i + seq_length +1] for i in idx]

    # Convert the input and output batches to tensors
    x_batch = torch.tensor(input_batch, dtype=torch.long)
    y_batch = torch.tensor(output_batch, dtype=torch.long)

    return x_batch, y_batch

# Perform some simple tests to make sure your batch function is working properly!
test_args = (vectorized_songs, 10, 2)
x_batch, y_batch = get_batch(*test_args)
assert x_batch.shape == (2, 10), "x_batch shape is incorrect"
assert y_batch.shape == (2, 10), "y_batch shape is incorrect"
print("Batch function works correctly!")


Batch function works correctly!


In [ ]:
x_batch, y_batch = get_batch(vectorized_songs, seq_length=5, batch_size=1)

for i, (input_idx, target_idx) in enumerate(zip(x_batch[0], y_batch[0])):
    print("Step {:3d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx.item()])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx.item()])))

Defining the RNN Model

In [35]:
# defining the model 
class lstmmodel(nn.Module):
    def __init__(self,vocab_size,embedding_dim , hidden_size):
        super(lstmmodel ,self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(vocab_size , embedding_dim)

        self.lstm = nn.LSTM(embedding_dim , hidden_size , batch_first = True)

        self.fc = nn.Linear(hidden_size , vocab_size)

    def init_hidden (self,batch_size,device) :
        #initialize both hidden state and cell state
        return (torch.zeros(1,batch_size,self.hidden_size).to(device),
                torch.zeros(1,batch_size,self.hidden_size).to(device))
    def forward(self, x , state = None , return_state = False ):
        x = self.embedding(x)

        if state is None :
            state = self.init_hidden(x.size(0) , x.device)
        out,state = self.lstm(x , state)

        out = self.fc(out)
        return out if not return_state else (out,state)

In [39]:
vocab_size = len(vocab)
embedding_dim = 256 
hidden_size = 1024
batch_size = 8

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
model = lstmmodel(vocab_size,embedding_dim,hidden_size).to(device)

print(model)

Using device: cpu
lstmmodel(
  (embedding): Embedding(83, 256)
  (lstm): LSTM(256, 1024, batch_first=True)
  (fc): Linear(in_features=1024, out_features=83, bias=True)
)


#### Testing the data

In [43]:
x , y = get_batch(vectorized_songs , seq_length=100 , batch_size=32)
x = x.to(device)
y = y.to(device)

pred = model(x)
print(f"Input shape :  {x.shape} ")
print(f"prediction shape :   {pred.shape}")

Input shape :  torch.Size([32, 100]) 
prediction shape :   torch.Size([32, 100, 83])


In [44]:
sampled_indices = torch.multinomial(torch.softmax(pred[0], dim=-1), num_samples=1)
sampled_indices = sampled_indices.squeeze(-1).cpu().numpy()
sampled_indices

array([ 5, 81, 67, 23, 27, 56, 76, 68,  6, 30, 37, 35, 80, 45, 42, 34,  6,
       55, 32, 24, 25, 79, 42, 56, 68, 63, 75, 24, 12, 41, 13,  7, 20, 46,
       33, 34, 19,  8, 58, 15, 18, 57, 15, 66, 42, 78, 11, 17, 29, 65, 59,
       11, 19, 72, 28, 26, 34, 30,  5, 70, 52, 23, 44,  3, 29, 30, 30, 55,
        0, 46,  4,  4, 34, 12, 25, 14, 29, 67, 41, 17, 48, 62, 15, 66, 77,
       77, 52, 43, 71, 69, 76,  8, 12, 30,  3, 15, 48, 80, 11, 53])

In [45]:
print("Input: \n", repr("".join(idx2char[x[0].cpu()])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices])))

Input: 
 'Bde d2Bd|e2eg e3g|!\nG2 (3GAB A2 (3ABc|B2 (3cde d3d|efgf edBd|e2eg e2ga|!\nbabg agae|gfgd edcB|cBcd ef'

Next Char Predictions: 
 '\'zl<Baum(ELJyTQI(_G=>xQamht=0P1)8UHI7,c36b3kQw/5Djd/7qCAIE\'o[<S"DEE_\nU##I0>2DlP5Wg3kvv[Rpnu,0E"3Wy/]'


#### Training the model and Loss checking operation